## Get list file

In [10]:
import os
import fnmatch
import pandas as pd

path = "D:/Cinémathèque/"  # F:\Data\Cinémathèque
pattern_list = ["*.mkv", "*.mp4"]

In [2]:
def get_pattern(file:str, list_pattern:list) -> str:
    i = 0
    while i < len(list_pattern):
        if fnmatch.fnmatch(file, list_pattern[i]):
            return list_pattern[i]
        else:
            i += 1
    return list_pattern[0]

In [3]:
def get_mkv_list(path="D:/Cinémathèque/", list_pattern=["*.mkv"]) -> pd.DataFrame:
    file_list = []
    for root, dirs, files in os.walk(path):
        if root == path:  # To prevent subfolder analyze
            for file in files:
                pattern = get_pattern(file, list_pattern)
                if fnmatch.fnmatch(file, pattern):  # to get only with ".mkv" extension
                    file_list.append({'path': path, 'file': file[:-len(pattern)+1], 'extension': pattern[1:]})
    return pd.DataFrame(file_list)

file_list = get_mkv_list()
file_list.head()

,path,file,extension
0,D:/Cinémathèque/,A Dangerous Method (2011) David Cronenberg,.mkv
1,D:/Cinémathèque/,"Aguirre, la colère de Dieu (1972) Werner Herzog",.mkv
2,D:/Cinémathèque/,All the President's Men (1976) Alan J Pakula,.mkv
3,D:/Cinémathèque/,April Story (1998) Shunji Iwai,.mkv
4,D:/Cinémathèque/,Après la tempete (2016) Hirokazu Kore-eda,.mkv


## Search a file

In [4]:
def search_file(file_list, query):
    return file_list[file_list.file.str.contains(query)].reset_index(drop=True)

res_file_list = search_file(file_list, "2018")
res_file_list

,path,file,extension
0,D:/Cinémathèque/,Capharnaum (2018) Nadine Labaki,.mkv
1,D:/Cinémathèque/,Grace à Dieu (2018) François Ozon,.mkv
2,D:/Cinémathèque/,"Les Oiseaux de passage (2018) Ciro Guerra, Cri...",.mkv
3,D:/Cinémathèque/,Leto (2018) Kirill Serebrennikov,.mkv
4,D:/Cinémathèque/,The House That Jack Built (2018) Lars von Trier,.mkv
5,D:/Cinémathèque/,Woman at War (2018) Benedikt Erlingsson,.mkv


## Torrent creation

In [5]:
from torf import Torrent

tracker_list = ['udp://tracker.opentrackr.org:1337/announce']

In [6]:
def create_torrent(path, file, extension, tracker_list=['udp://tracker.opentrackr.org:1337/announce']):
    if os.path.exists(path + file + '.torrent'):
        return path + file + '.torrent', file + '.torrent already exists !'
    
    try:
        t = Torrent(path     = path + file + extension,
                    trackers = tracker_list,
                    comment  = 'Have fun !')
        t.private = True
        t.generate()
        t.write(path + file + '.torrent')
    except Exception as e:
        return None, 'Error during torrent creation (wrong file name ?):\n' + str(e)
    
    return path + file + '.torrent', file + '.torrent is created !'
    
# created_torrent_path, create_torrent_message = create_torrent(res_file_list.path.at[0], res_file_list.file.at[0], res_file_list.extension.at[0], tracker_list=['udp://tracker.opentrackr.org:1337/announce'])
# print(create_torrent_message + ' (' + created_torrent_path + ')')

## Add torrent to deluge

In [2]:
# import transmission
import clutch

# client = Client(address="http://localhost:9091/transmission/rpc")

In [7]:
# deluge.add_torrent_file(created_torrent_path)
def add_torrent_list(torrent_list: list):
    client = clutch.Client(address="http://localhost:9091/transmission/rpc")
    
    for torrent in torrent_list:
        client.torrent.add(arguments={
            "filename": torrent['path'] + torrent['file'],
            "download_dir": torrent['path'],
            "paused": False
          })
        
        
torrent_list = [{'path': "D:/Cinémathèque/", 'file': 'Psycho Goreman (2020) Steven Kostanski.torrent'}, {'path': "D:/Cinémathèque/", 'file': 'Saint Maud (2020) Rose Glass.torrent'}]
# add_torrent_list(torrent_list)

# Main

In [11]:
movies = [
    'Sound of Metal', 'Judas and the Black Messiah', 'Minari', 
    'Nomadland', 'Promising Young Woman', 'Possessor', 'Come True', 
    'Malcolm & Marie', 'Les Sept de Chicago', 'Le Peuple Loup', 
    'Mank', 'The Father', 'Saint Maud', 'Psycho Goreman']

files = get_mkv_list(path="D:/Cinémathèque/", list_pattern=pattern_list)

In [12]:
for movie in movies:
    res = search_file(files, movie)
    print(res)
    create_torrent(res.path.at[0], res.file.at[0], res.extension.at[0], tracker_list=['udp://tracker.opentrackr.org:1337/announce'])
    
print('\nCongrat\'')

               path                                 file extension
0  D:/Cinémathèque/  Sound of Metal (2020) Darius Marder      .mkv
               path                                           file extension
0  D:/Cinémathèque/  Judas and the Black Messiah (2021) Shaka King      .mkv
               path                           file extension
0  D:/Cinémathèque/  Minari (2020) Lee Isaac Chung      .mkv
               path                         file extension
0  D:/Cinémathèque/  Nomadland (2021) Chloé Zhao      .mkv
               path                                          file extension
0  D:/Cinémathèque/  Promising Young Woman (2021) Emerald Fennell      .mkv
               path                                 file extension
0  D:/Cinémathèque/  Possessor (2020) Brandon Cronenberg      .mkv
               path                                  file extension
0  D:/Cinémathèque/  Come True (2021) Anthony Scott Burns      .mkv
               path                               

---

Thibault **Santonja**  
2021